In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import time

In [6]:
paths = ['dataset/']

In [7]:
def imshow(img):
    fig, ax = plt.subplots(1,1, figsize=(8,8))
    ax.imshow(img, cmap="gray")

In [10]:
possible_labels = os.listdir(os.path.join(paths[0]))
possible_labels

['closed', 'open']

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,  # Normalización de píxeles (0 a 1)
    brightness_range=[0.5, 1.5],  # Ajuste de brillo (reduce o aumenta iluminación)
    zoom_range=0.2,  # Zoom aleatorio (acerca o aleja)
    horizontal_flip=True,  # Volteo horizontal
    preprocessing_function=lambda x: cv.GaussianBlur(x, (5, 5), 0)
)


Found 2679 images belonging to 2 classes.


In [45]:
import numpy as np
import cv2 as cv
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Definir Data Augmentation
datagen = ImageDataGenerator(
    brightness_range=[0.5, 1.5],  # Cambio de brillo
    zoom_range=0.2,  # Zoom
    horizontal_flip=True,  # Volteo horizontal
    preprocessing_function=lambda x: cv.GaussianBlur(x.squeeze(), (5, 5), 0)[..., np.newaxis] # Desenfoque
)

data = []

for i, path in enumerate(paths):
    for label_int, label_string in enumerate(possible_labels):
        filenames = glob.glob(path + label_string + '/*.jpg')

        for filename in filenames:
            img = cv.imread(filename, cv.IMREAD_GRAYSCALE)
            img = cv.resize(img, (224, 224))

            # Agregar canal para compatibilidad con Keras
            img = np.expand_dims(img, axis=-1)  # (224, 224) → (224, 224, 1)

            # Hacer batch de 1 imagen para ImageDataGenerator
            img_batch = np.expand_dims(img, axis=0)  # (224, 224, 1) → (1, 224, 224, 1)

            # Aplicar Data Augmentation
            it = datagen.flow(img_batch, batch_size=1)

            # Agregar imagen original
            data.append([label_int,img.squeeze()])

            # Generar 5 imágenes aumentadas
            for _ in range(5):  # Generar 5 aumentaciones por imagen
                augmented_img = it.next()[0]  # Obtener imagen aumentada
                data.append([label_int, augmented_img.squeeze()])  # Guardar imagen aumentada

print(f"Total de imágenes en el dataset: {len(data)}")

Total de imágenes en el dataset: 16074


In [46]:
import random
random.Random(0).shuffle(data)

x_train = []
y_train = []
x_test = []
y_test = []

for i, sample in enumerate(data):
    label, img = sample[0], sample[1]
    if i <= 0.7 * len(data):
        x_train.append(img)
        y_train.append(label)
    else:
        x_test.append(img)
        y_test.append(label)

x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [47]:
x_train = x_train/255
x_test = x_test/255

In [48]:
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [49]:
print(x_train.shape)
print(x_test.shape)

(11252, 224, 224)
(4822, 224, 224)


In [50]:
print(y_train.shape)
print(y_test.shape)

(11252, 2)
(4822, 2)


# Red neuronal

In [54]:
def model():
    model = Sequential()
    model.add(Conv2D(32,(2,2), input_shape=(224, 224, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Conv2D(64,(2,2), input_shape=(224, 224, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(2,2), input_shape=(224, 224, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model

In [55]:
model = model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 223, 223, 32)      160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 73, 73, 64)        8256      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 35, 35, 128)       32896     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 128)     

In [56]:
model.fit(x_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
176/176 [==============================] - 21s 38ms/step - loss: 0.2309 - accuracy: 0.8946 - auc_1: 0.9687 - precision_1: 0.8945 - recall_1: 0.8918
Epoch 2/10
176/176 [==============================] - 6s 35ms/step - loss: 0.0474 - accuracy: 0.9845 - auc_1: 0.9979 - precision_1: 0.9846 - recall_1: 0.9842
Epoch 3/10
176/176 [==============================] - 6s 34ms/step - loss: 0.0229 - accuracy: 0.9923 - auc_1: 0.9996 - precision_1: 0.9921 - recall_1: 0.9923
Epoch 4/10
176/176 [==============================] - 6s 34ms/step - loss: 0.0141 - accuracy: 0.9957 - auc_1: 0.9998 - precision_1: 0.9956 - recall_1: 0.9958
Epoch 5/10
176/176 [==============================] - 6s 35ms/step - loss: 0.0135 - accuracy: 0.9964 - auc_1: 0.9997 - precision_1: 0.9964 - recall_1: 0.9962
Epoch 6/10
176/176 [==============================] - 6s 34ms/step - loss: 0.0096 - accuracy: 0.9971 - auc_1: 0.9999 - precision_1: 0.9972 - recall_1: 0.9969
Epoch 7/10
176/176 [==============================]

In [57]:
model.evaluate(x_test, y_test)

NotFoundError: Graph execution error:

Detected at node 'sequential_1/conv2d_3/Relu' defined at (most recent call last):
    File "c:\Users\ulise\anaconda3\envs\py310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\ulise\anaconda3\envs\py310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\ulise\anaconda3\envs\py310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\ulise\anaconda3\envs\py310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\ulise\anaconda3\envs\py310\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\ulise\anaconda3\envs\py310\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\ulise\anaconda3\envs\py310\lib\

In [58]:
p = 1
q = p + 1

for i in range(10):
    prediction = model.predict(x_test[p:q])
    prediction_label = prediction.argmax()
    print(f'Prediction: {prediction_label}')
    print(f'Etiqueta real: {y_test[p].argmax()}')
    p += 1
    q = p+1
    time.sleep(1)

1/1 [==============================] - 0s 231ms/step
Prediction: 0
Etiqueta real: 0
1/1 [==============================] - 0s 20ms/step
Prediction: 0
Etiqueta real: 0
1/1 [==============================] - 0s 19ms/step
Prediction: 0
Etiqueta real: 0
1/1 [==============================] - 0s 22ms/step
Prediction: 0
Etiqueta real: 0
1/1 [==============================] - 0s 19ms/step
Prediction: 0
Etiqueta real: 0
1/1 [==============================] - 0s 19ms/step
Prediction: 1
Etiqueta real: 1


KeyboardInterrupt: 

In [59]:
model.save("models/open-closed-eyes-v12.h5")